In [7]:
!cd /app/libraries && python3 -m pip install -r requirements.txt

Obtaining file:///app/libraries/slib (from -r requirements.txt (line 1))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
Obtaining file:///app/libraries/brdf_enc_dec (from -r requirements.txt (line 2))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
Obtaining file:///app/libraries/staf (from -r requirements.txt (line 3))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for slib (pyproject.toml) ... done
  Created wheel for slib: filename=slib-2023.4.18-0.editable-py3-none-any.whl size=2620 sha256

In [1]:
from pathlib import Path
class Settings:
    dataset_dir = Path("/app/resources/")
settings = Settings()    

In [4]:
# import sys
# sys.path.append("/app/libraries")

import argparse

import numpy as np
import torch
from tqdm import tqdm
from brdf_enc_dec.torrance_sparrow_dataset import TorspaBRDFNPsDataset
from brdf_enc_dec.model import BRDFNPs,ModelConfig
from staf import STAF
from TorranceSparrow import TorranceSparrowParams,torspa
from pathlib import Path

class Settings:
    dataset_dir = Path("/app/resources/")
    enc_dec_checkpoint = dataset_dir /"model_weights/encdec/600k.pth"
settings = Settings()    
# parser = argparse.ArgumentParser()
# parser.add_argument("--tvBTF",type=int,default=45)
# parser.add_argument("--poly",type=int,default=3)
# args = parser.parse_args()
class args:
    tvBTF = 45
    poly = 3
    batch_size = 128
    n_frames = 36 # number of frames to use        
    device="-1"
    out_dir = Path("./output")

device = "cuda" if args.device != "-1" else "cpu"    


# load brdf encoder/decoder
brdfnps=BRDFNPs(device=device,checkpoint=settings.enc_dec_checkpoint,onlyEncoder=True,cfg=ModelConfig())
brdfnps.to_eval()


# load tvBRDF data
staf = STAF.parse_files(
    settings.dataset_dir/f"tvBTF{args.tvBTF}/poly-{args.poly}"
)
latents_all=[]
t=0
frame=0
# output directory
out_dir = args.out_dir/ f"t-{t:.2f}" 
workspace_dir = out_dir / "batch_output"
workspace_dir.mkdir(exist_ok=True,parents=True)


tsp = staf.get_tsp(t=t,device=device)
dataset = TorspaBRDFNPsDataset(params = tsp, batch_size=args.batch_size, device=device)
results=[]


with torch.no_grad():
    for i in tqdm(range(dataset.num_baches)):
        batch_input = dataset.get_batch(i).reshape(-1, 16200, 7)
        latent = brdfnps.encode(batch_input)
        results.append(latent.cpu().numpy())
        np.save(
            workspace_dir / f"batch_{i}.npy",
            latent.cpu().numpy()
        )
    results = np.concatenate(results, axis=0)
    np.save(
        out_dir / f"f{frame}_latents_all.npy",
        results
    )
    latents_all.append(results[..., :ModelConfig().latent_dim].reshape(-1, 1, 10))

model path: /app/resources/model_weights/encdec/600k.pth
Set up models...
device: cpu
loaded


/app/libraries/brdf_enc_dec/brdf_enc_dec/torrance_sparrow_dataset.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params.kd = torch.tensor(self.params.kd, dtype=torch.float32).to(
/app/libraries/brdf_enc_dec/brdf_enc_dec/torrance_sparrow_dataset.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params.ks = torch.tensor(self.params.ks, dtype=torch.float32).to(
/app/libraries/brdf_enc_dec/brdf_enc_dec/torrance_sparrow_dataset.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params.sigma = torch.tensor(


device cpu


  0%|          | 0/1654 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [5]:
np.save(args.out_dir / f"latents_all.npy", np.concatenate(latents_all, axis=1))

10